# Coin Accounter System 

In [1]:
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot as plt
import numpy as np
import cv2 as cv
import glob
import time

/home/rogeriojunior/gpam/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/rogeriojunior/gpam/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/rogeriojunior/gpam/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/rogeriojunior/gpam/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/rogeriojunior/gpam/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy

In [2]:
# Função para facilitar a visualização das imagens
def showit(img, openCV=0):
    
    if(openCV):
        cv.imshow("t", img)
        cv.waitKey(0)
        cv.destroyAllWindows()
    
    else:  
        # Para imagens preto e branco
        if(len(img.shape) < 3):
            plt.gray()
            plt.imshow(img)
            plt.show()
        else:
            plt.imshow(img[:,:,::-1])
            plt.show()

## LBP Function to Extract Pattern in Coins

In [3]:
def lbp(img):
    qtdeLinhas, qtdeColunas = img.shape
    out = np.zeros((qtdeLinhas,qtdeColunas), dtype=int)
    
    for i in range(1,qtdeLinhas-1):
        for j in range(1,qtdeColunas-1):
            a=img[i-1,j]
            b=img[i-1,j+1]
            c=img[i,j+1]
            d=img[i+1,j+1]
            e=img[i+1,j]
            f=img[i+1,j-1]
            g=img[i,j-1]
            h=img[i-1,j-1]
            centro=img[i,j]
            
            soma=0
            
            if(a<centro):
                soma = soma + 2**7
            if(b<centro):
                soma = soma + 2**6
            if(c<centro):
                soma = soma + 2**5
            if(d<centro):
                soma = soma + 2**4
            if(e<centro):
                soma = soma + 2**3
            if(f<centro):
                soma = soma + 2**2
            if(g<centro):
                soma = soma + 2**1
            if(h<centro):
                soma = soma + 2**0

            out[i,j]=soma  
            
    return out

def histograma(img):
    qtdeLinhas, qtdeColunas= img.shape
    w = np.zeros(256)
    
    for i in range(qtdeLinhas):
        for j in range(qtdeColunas):
            w[img[i,j]] = w[img[i,j]]+1
    for i in range(256):
        w[i] = w[i]/(qtdeLinhas*qtdeColunas)
        
    return w

In [4]:
def coloredHistogram(img):
    qtdeLinhas, qtdeColunas, c = img.shape
    wb = np.zeros(256)
    wg = np.zeros(256)
    wr = np.zeros(256)
    w = np.zeros(256)
    
    color=0
    
    for i in range(qtdeLinhas):
        for j in range(qtdeColunas):
            wb[img[i,j,0]] = wb[img[i,j,0]] + 1
            wg[img[i,j,1]] = wg[img[i,j,1]] + 1
            wr[img[i,j,2]] = wr[img[i,j,2]] + 1
            
    for i in range(256):
        w[i] = min(wb[i], wg[i], wr[i])
        
    for i in range(256):
        w[i] = w[i]/(qtdeLinhas*qtdeColunas)
        
    return w

In [5]:
def saturationHistogram(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    
    qtdeLinhas, qtdeColunas,c= img.shape
    w = np.zeros(256)
    saturation=0
    
    for i in range(qtdeLinhas):
        for j in range(qtdeColunas):
            w[img[i,j,1]] = w[img[i,j,1]]+1
    for i in range(256):
        w[i] = w[i]/(qtdeLinhas*qtdeColunas)
        
    return w

### Improvements in coins

In [6]:
def improveEdges(imgCol):
    
    # Detecção de bordas
    sobelVert = cv.Sobel(imgCol, cv.CV_8U, 1, 0, ksize=3)
    sobelHori = cv.Sobel(imgCol, cv.CV_8U, 0, 1, ksize=3)
    imgColSobel = cv.add(sobelHori, sobelVert)
    
    # Realçando as bordas encontradas na imagem colorida
    stt = cv.getStructuringElement(cv.MORPH_ELLIPSE, (2,2))
    imgColSobel = cv.morphologyEx(imgColSobel, cv.MORPH_CLOSE, stt)
    
    # Adicionar as bordas encontradas na imagem colorida
    imgCol = cv.add(imgCol, imgColSobel)
#     showit(imgCol, 1)
    
    return imgCol


def improveTexture(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img = clahe.apply(img)
    
    return img


def resize(img, cols=800):
    
    d = cols / img.shape[1]
    dim = (cols, int(img.shape[0] * d))
    img = cv.resize(img, dim, interpolation=cv.INTER_AREA)
    
    return img

### Extract coin out of background

In [7]:
# Função que retorna a mascara das moedas na imagem
def getMask(imgCol):
    
    imgGray = cv.cvtColor(imgCol, cv.COLOR_BGR2GRAY)
    
    # Encontrar a região de interesse (mascara da região da moeda)
    limiar, imgGrayOTSU = cv.threshold(imgGray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU) # limiar de otsu inclui as sombras
    ret,imgGray = cv.threshold(imgGray,limiar ,255,cv.THRESH_BINARY_INV) # 85% do limiar de otsu
    
#     showit(imgGray,1)
    
    # Parâmetros para selecionar só a região de interesse
    val = 3
    maskSize=5
    
#     # Remover pequenas falhas antes de dilatar
#     stt = cv.getStructuringElement(cv.MORPH_ELLIPSE, (maskSize,maskSize))
#     imgBin = cv.erode(imgGray, stt, iterations=1)
#     # showit(imgBin, 1)

#     # Fechar buracos na estrutura encontrada
#     stt = cv.getStructuringElement(cv.MORPH_ELLIPSE, (maskSize,maskSize))
#     imgBin = cv.morphologyEx(imgBin, cv.MORPH_CLOSE, stt)
#     imgBin = cv.dilate(imgBin, stt, iterations=val+30)

# #     showit(imgBin, 1)

#     # Erosão para remover pequenas falhas e separar moedas coladas
#     stt = cv.getStructuringElement(cv.MORPH_ELLIPSE, (maskSize,maskSize))
#     imgBin = cv.erode(imgBin, stt, iterations=val)

    stt = cv.getStructuringElement(cv.MORPH_ELLIPSE, (maskSize,maskSize))
    imgBin = cv.morphologyEx(imgGray, cv.MORPH_CLOSE, stt, iterations=20)

    showit(imgBin, 1)

    return imgBin


# Função que encontrar a moeda na imagem e retorna o menor quadrado envolvente com a moeda
def findSingleCoin(imgCol):
    
    # Reduz tamanho da imagem
    imgCol = resize(imgCol, 800)
    
    a=4
    b=4
    kernel = np.array([[-1/a,-1/a,-1/a],[-1/a,b,-1/a],[-1/a,-1/a,-1/a]])
    imgCol = cv.filter2D(imgCol,-1,kernel)
        
    imgBin = getMask(imgCol)    

    # Convertendo imgGray para BGR para poder realizar o bitwise
    imgGray = cv.cvtColor(imgBin, cv.COLOR_GRAY2BGR)

    # Selecionando região de interesse na imagem colorida com a máscara encontrada
    imgCol = cv.bitwise_and(imgCol, imgGray)
    
#     showit(imgCol, 1)
    
#     imgCol = improveEdges(imgCol)

    showit(imgCol, 1)
    
    # Achar Contornos das moedas
    moedas, hierarquia = cv.findContours(imgBin, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        
    # Para cada moeda encontrada
    for moeda in moedas:
        
        # Não contabilizar regiões de falhas
        area = cv.contourArea(moeda)
        if(area < 10000.0):
            continue
            
        # Encontre o menor retângulo envolvente
        x, y, w, h = cv.boundingRect(moeda)

        # Nova imagem com o menor retângulo envolvente da moeda
        imgCoinRoi = imgCol[y:y+h, x:x+w, ::-1]
        
#         showit(imgCoinRoi, 1)

    return imgCoinRoi

### Extractions Functions Routines

In [8]:
def extraction(imag, roiDefined=False):   
    
    if(not roiDefined):
        imag = findSingleCoin(imag)

    color = coloredHistogram(imag)
    saturation = saturationHistogram(imag)

    imag = improveTexture(imag)
    imag = lbp(imag)
    texture = histograma(imag)
    
    return np.append(np.append(texture, color), saturation)

def extractionFromFile(file):
    img = cv.imread(file)
    return extraction(img)

## Set Train Dataset

In [ ]:
start_time = time.time()

# Essa função realça as bordas da imagem

class Enum(tuple):__getattribute__ = tuple.index

# ETAPA 1: EXTRAIR CARACTERÍSTICAS DAS IMAGENS
Coins = Enum(("CINCO", "DEZ", "VINTECINCO", "CINQUENTA", "REAL"))

coin_directory = "moedas/"

# Lista com o nome das imagens nesse diretório
sample_images_5_back = glob.glob(coin_directory + "5/back/*")
sample_images_5_front = glob.glob(coin_directory + "5/front/*")

sample_images_10_back = glob.glob(coin_directory  + "10/back/*")
sample_images_10_front = glob.glob(coin_directory + "10/front/*")

sample_images_25_back = glob.glob(coin_directory  + "25/back/*")
sample_images_25_front = glob.glob(coin_directory + "25/front/*")

sample_images_50_back = glob.glob(coin_directory  + "50/back/*")
sample_images_50_front = glob.glob(coin_directory + "50/front/*")

sample_images_100_back = glob.glob(coin_directory  + "100/back/*")
sample_images_100_front = glob.glob(coin_directory + "100/front/*")

# Lista com as características das imagens e suas respectivas classificações
data = []
answrData = []
arffData = []
classes = ['0', '1', '2', '3', '4']

# # Extraindo características das moedas sozinhas
# for i in sample_images_5_front:
#     features = extractionFromFile(i)
#     data.append(features)
#     answrData.append(Coins.CINCO)
    
#     arffData.append((features, str(Coins.CINCO)))

# for i in sample_images_5_back:
#     features = extractionFromFile(i)
#     data.append(features)
#     answrData.append(Coins.CINCO)
    
#     arffData.append((features, str(Coins.CINCO)))

# for i in sample_images_10_front:
#     features = extractionFromFile(i)
#     data.append(features)
#     answrData.append(Coins.DEZ)
    
#     arffData.append((features, str(Coins.DEZ)))

# for i in sample_images_10_back:
#     features = extractionFromFile(i)
#     data.append(features)
#     answrData.append(Coins.DEZ)
    
#     arffData.append((features, str(Coins.DEZ)))

# for i in sample_images_25_front:
#     features = extractionFromFile(i)
#     data.append(features)
#     answrData.append(Coins.VINTECINCO)
    
#     arffData.append((features, str(Coins.VINTECINCO)))

# for i in sample_images_25_back:
#     features = extractionFromFile(i)
#     data.append(features)
#     answrData.append(Coins.VINTECINCO)
    
#     arffData.append((features, str(Coins.VINTECINCO)))

# for i in sample_images_50_front:
#     features = extractionFromFile(i)
#     data.append(features)
#     answrData.append(Coins.CINQUENTA)
    
#     arffData.append((features, str(Coins.CINQUENTA)))

# for i in sample_images_50_back:
#     features = extractionFromFile(i)
#     data.append(features)
#     answrData.append(Coins.CINQUENTA)
    
#     arffData.append((features, str(Coins.CINQUENTA)))

for i in sample_images_100_front:
    features = extractionFromFile(i)
    data.append(features)
    answrData.append(Coins.REAL)
    
    arffData.append((features, str(Coins.REAL))) 

for i in sample_images_100_back:
    features = extractionFromFile(i)
    data.append(features)
    answrData.append(Coins.REAL)
    
    arffData.append((features, str(Coins.REAL)))

print("--- %s seconds ---" % (time.time() - start_time))

## Classifier

In [10]:
clf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=1).fit(data,answrData)

## Data Input

In [41]:
def predictCoin(roi):
    hist = extraction(roi, True)
    s = clf.predict([hist])
    
    return Coins[int(s)]

def findMultiplesCoin(imgCol):
    img = cv.cvtColor(imgCol, cv.COLOR_BGR2GRAY)
    img = improveEdges(img)
    img = cv.blur(img, (3,3))

    # Single channel to multiples channel
    cimg = cv.cvtColor(img,cv.COLOR_GRAY2BGR)

    circles = cv.HoughCircles(img,method=cv.HOUGH_GRADIENT,dp=1,minDist=100,
                               param1=50,param2=50,minRadius=40,maxRadius=80)
    return circles

# Imagem que será analisada
img = cv.imread("moedas/juntas/10front-10front-5back-5back-100front-100front-50front-50back-25back-25back-5back-5front-10front-10front-5front-5front-5front-50back.jpg")
img = resize(img, 800)
a=4
b=4
kernel = np.array([[-1/a,-1/a,-1/a],[-1/a,b,-1/a],[-1/a,-1/a,-1/a]])
img = cv.filter2D(img,-1,kernel)
 


# Cópia da imagem original que será apresentada no final
output = img.copy()

circles = findMultiplesCoin(img)

count = 0
if circles is not None:
    # coordinates and radii
    circles = np.round(circles[0,:]).astype(int)
    
    for (x, y, d) in circles:
        count += 1
                
        roi = img[y-d:y+d, x-d:x+d]
        
        roi = improveEdges(roi)
        
        coin = predictCoin(roi)
        
        cv.circle(output, (x,y), d, (0, 255, 0), 2)
        cv.putText(output, coin, (x - 40, y), cv.FONT_HERSHEY_PLAIN,
                   1.5, (0, 255, 0), thickness=2, lineType=cv.LINE_AA)

showit(output, True)

# TODO
> normalizar imagem

> remover sombras

> ajustar circulo das moedas

## Arquivo ARFF para testar no Keras